<a href="https://colab.research.google.com/github/acevedosharp/ensemble-testing-chamber/blob/master/ensemble_classifier_combination_tester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- assemble all possible combination of ensembles $k \in \{1,2,3,4,5\}$
- 10-fold cross validation


In [13]:
import itertools
import numpy as np
import sklearn
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# classifiers
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# datasets
from sklearn.datasets import *

from datetime import datetime

In [ ]:
classifiers = {
    "Linear SVC": LinearSVC(),
    "Decission Tree": DecisionTreeClassifier(),
    "Extra Tree": ExtraTreeClassifier(),
    "Logistic": LogisticRegression(),
    "Passive Aggressive": PassiveAggressiveClassifier(),
    "Perceptron": Perceptron(),
    "Ridge": RidgeClassifier(),
    "SGD": SGDClassifier(),
    "Multi-layer Perceptron": MLPClassifier(),
    "Linear Discriminant": LinearDiscriminantAnalysis(),
    "Quadratic Discriminant": QuadraticDiscriminantAnalysis(),
    "BernoulliNB": BernoulliNB(),
    "MultinomialNB": MultinomialNB(),
    "Nearest Neighbors": KNeighborsClassifier(),
    "Extra Trees": ExtraTreesClassifier(),
    "Random Forest (10 estimators)": RandomForestClassifier(n_estimators=10),
    "Gradient Boosting": GradientBoostingClassifier()
}

ds_names = [
            "Digits",
            "Iris",
            "Wine",
            "Breast Cancer"
]

datasets = [
            sklearn.datasets.load_breast_cancer(return_X_y=True),
            # sklearn.datasets.load_iris(return_X_y=True),
            # sklearn.datasets.load_wine(return_X_y=True),
            sklearn.datasets.load_digits(return_X_y=True)
]

results = []

kf = KFold(n_splits=10)
for ds_idx, ds in enumerate(datasets):
  print("==================== CHANGING DATASET at ", datetime.now(), "====================")
  X, Y = ds[0], ds[1]
  #X = StandardScaler().fit_transform(X)
  fold_index = 0
  for train_index, test_index in kf.split(X):
    print("==================== CHANGING FOLD at ", datetime.now(), "====================")
    X_train, X_test = X[train_index], X[test_index] # np.take
    Y_train, Y_test = Y[train_index], Y[test_index] # np.take

    # Train every classifier with the new data
    print("++++++++++ began training classifiers at", datetime.now(), "++++++++++")
    for classifier_name, classifier in classifiers.items():
      classifier.fit(X_train, Y_train)
    print("++++++++++ ended training classifiers at", datetime.now(), "++++++++++")
    # Assemble ensembles of size k in {1,2,3,4,5}
    for k in range(1,6):
      print("==================== CHANGING to k = ", k, " at ", datetime.now(), "====================")
      for combination in list(itertools.combinations(classifiers.keys(), k)):
        ensemble = []
        ensemble_description = ""

        # group classifiers (already exist fitted in dict)
        for idx in range(k):
          ensemble.append(classifiers[combination[idx]])
          ensemble_description += combination[idx]
          ensemble_description += ","
        ensemble_description = ensemble_description[:-1]

        # save predictions
        predictions = np.zeros((len(X_test), k)) # (# test instances, ensemble size)
        for idx, classifier in enumerate(ensemble):
          predictions[:,idx] = classifier.predict(X_test)
        
        # do hard voting
        hard_voting_predictions = np.zeros((len(X_test), 1)) # (# test instances, 1)
        for idx in range(predictions.shape[0]):
          values, counts = np.unique(predictions[idx], return_counts=True)
          hard_voting_predictions[idx] = values[np.argmax(counts)]
        
        # compare voting predictions against Y_test
        total_instance_number = len(X_test)
        errors = 0
        for idx in range(hard_voting_predictions.shape[0]):
          if (hard_voting_predictions[idx][0] != Y_test[idx]):
            errors += 1
        score = errors/total_instance_number

        # save result (ensemble description, ensemble size, fold index, dataset, score)
        results.append([ensemble_description, k, fold_index, ds_names[ds_idx], score])

    fold_index += 1
        

==================== CHANGING DATASET at  2021-02-05 11:13:16.191461 ====================
==================== CHANGING FOLD at  2021-02-05 11:13:16.192352 ====================
++++++++++ began training classifiers at 2021-02-05 11:13:16.192785 ++++++++++


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


++++++++++ ended training classifiers at 2021-02-05 11:13:17.573494 ++++++++++
==================== CHANGING to k =  1  at  2021-02-05 11:13:17.574008 ====================
==================== CHANGING to k =  2  at  2021-02-05 11:13:17.621208 ====================
==================== CHANGING to k =  3  at  2021-02-05 11:13:18.129100 ====================
==================== CHANGING to k =  4  at  2021-02-05 11:13:21.105273 ====================
==================== CHANGING to k =  5  at  2021-02-05 11:13:34.116641 ====================
==================== CHANGING FOLD at  2021-02-05 11:14:14.344531 ====================
++++++++++ began training classifiers at 2021-02-05 11:14:14.344765 ++++++++++


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


++++++++++ ended training classifiers at 2021-02-05 11:14:15.556557 ++++++++++
==================== CHANGING to k =  1  at  2021-02-05 11:14:15.556744 ====================
==================== CHANGING to k =  2  at  2021-02-05 11:14:15.603991 ====================
==================== CHANGING to k =  3  at  2021-02-05 11:14:16.115421 ====================
==================== CHANGING to k =  4  at  2021-02-05 11:14:19.264292 ====================
==================== CHANGING to k =  5  at  2021-02-05 11:14:32.408528 ====================
==================== CHANGING FOLD at  2021-02-05 11:15:12.658178 ====================
++++++++++ began training classifiers at 2021-02-05 11:15:12.659088 ++++++++++


...

++++++++++ ended training classifiers at 2021-02-05 11:23:08.565320 ++++++++++
==================== CHANGING to k =  1  at  2021-02-05 11:23:08.566419 ====================
==================== CHANGING to k =  2  at  2021-02-05 11:23:08.752686 ====================
==================== CHANGING to k =  3  at  2021-02-05 11:23:10.837076 ====================
==================== CHANGING to k =  4  at  2021-02-05 11:23:24.350533 ====================
==================== CHANGING to k =  5  at  2021-02-05 11:24:22.307907 ====================
==================== CHANGING FOLD at  2021-02-05 11:27:23.903446 ====================
++++++++++ began training classifiers at 2021-02-05 11:27:23.904813 ++++++++++


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


++++++++++ ended training classifiers at 2021-02-05 11:27:37.177734 ++++++++++
==================== CHANGING to k =  1  at  2021-02-05 11:27:37.177936 ====================
==================== CHANGING to k =  2  at  2021-02-05 11:27:37.371801 ====================
==================== CHANGING to k =  3  at  2021-02-05 11:27:39.539705 ====================
==================== CHANGING to k =  4  at  2021-02-05 11:27:53.351009 ====================
==================== CHANGING to k =  5  at  2021-02-05 11:28:51.802026 ====================
==================== CHANGING FOLD at  2021-02-05 11:31:53.093803 ====================
++++++++++ began training classifiers at 2021-02-05 11:31:53.094901 ++++++++++


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


++++++++++ ended training classifiers at 2021-02-05 11:32:06.357711 ++++++++++
==================== CHANGING to k =  1  at  2021-02-05 11:32:06.358912 ====================
==================== CHANGING to k =  2  at  2021-02-05 11:32:06.549439 ====================
==================== CHANGING to k =  3  at  2021-02-05 11:32:08.734599 ====================
==================== CHANGING to k =  4  at  2021-02-05 11:32:22.518220 ====================
==================== CHANGING to k =  5  at  2021-02-05 11:33:20.899418 ====================


### Persist results in bulk to a database